# Python 下載XML檔案與解析


* 了解 xml 檔案格式與內容
* 能夠利用套件存取 xml 格式的檔案


## 作業目標

* 比較一下範例檔案中的「File I/O」與「xmltodict」讀出來的內容有什麼差異

* 根據範例檔案的結果：
    1. 請問高雄市有多少地區有溫度資料？
    2. 請取出每一個地區所記錄的第一個時間點跟溫度
    3. 請取出第一個地區所記錄的每一個時間點跟溫度

### 比較一下範例檔案中的「File I/O」與「xmltodict」讀出來的內容有什麼差異


In [8]:
# File I/O
with open("D:/data/example3/10020_Week24_EN.xml", "r", encoding="utf-8") as fh:
    xml = fh.read()
print(xml)

<?xml version="1.0" encoding="utf-8"?>
<cwbopendata xmlns="urn:cwb:gov:tw:cwbcommon:0.1">
  <identifier>1c23836b-32a7-43b5-aab3-b2909b5bebc4</identifier>
  <sender>weather@cwb.gov.tw</sender>
  <sent>2019-06-28T12:22:56+08:00</sent>
  <status>Actual</status>
  <scope>Public</scope>
  <msgType>Issue</msgType>
  <dataid>null</dataid>
  <source>MFC</source>
  <dataset>
    <datasetInfo>
      <datasetDescription>Seven Days Weather Forecast of 368 townships</datasetDescription>
      <datasetLanguage>EN</datasetLanguage>
      <issueTime>2019-06-28T11:00:00+08:00</issueTime>
      <validTime>
        <startTime>2019-06-28T12:00:00+08:00</startTime>
        <endTime>2019-07-06T11:00:00+08:00</endTime>
      </validTime>
      <update>2019-06-28T12:22:56+08:00</update>
    </datasetInfo>
    <locations>
      <locationsName>Chiayi City</locationsName>
      <location>
        <locationName>East District</locationName>
        <geocode>1002001</geocode>
        <lat>23.480384</lat>
        <l

In [14]:
# xmltodict
import xmltodict
with open ("D:/Data/example3/10020_Week24_EN.xml","r",encoding = "utf-8") as fh:
    doc = dict(xmltodict.parse(fh.read()))
print(doc)

{'cwbopendata': OrderedDict([('@xmlns', 'urn:cwb:gov:tw:cwbcommon:0.1'), ('identifier', '1c23836b-32a7-43b5-aab3-b2909b5bebc4'), ('sender', 'weather@cwb.gov.tw'), ('sent', '2019-06-28T12:22:56+08:00'), ('status', 'Actual'), ('scope', 'Public'), ('msgType', 'Issue'), ('dataid', 'null'), ('source', 'MFC'), ('dataset', OrderedDict([('datasetInfo', OrderedDict([('datasetDescription', 'Seven Days Weather Forecast of 368 townships'), ('datasetLanguage', 'EN'), ('issueTime', '2019-06-28T11:00:00+08:00'), ('validTime', OrderedDict([('startTime', '2019-06-28T12:00:00+08:00'), ('endTime', '2019-07-06T11:00:00+08:00')])), ('update', '2019-06-28T12:22:56+08:00')])), ('locations', OrderedDict([('locationsName', 'Chiayi City'), ('location', [OrderedDict([('locationName', 'East District'), ('geocode', '1002001'), ('lat', '23.480384'), ('lon', '120.445361'), ('weatherElement', [OrderedDict([('elementName', 'T'), ('description', 'average of temperature'), ('time', [OrderedDict([('startTime', '2019-06-2

### 根據範例檔案的結果：

1. 請問高雄市有多少地區有溫度資料？
2. 請取出每一個地區所記錄的第一個時間點跟溫度
3. 請取出第一個地區所記錄的每一個時間點跟溫度

In [18]:
# 1. 請問高雄市有多少地區有溫度資料？
import xmltodict
with open ("D:/Data/example3/64_72hr_CH.xml","r",encoding = "utf-8") as fh:
    doc = dict(xmltodict.parse(fh.read()))


In [21]:
import pandas as pd 
locations = doc['cwbopendata']['dataset']['locations']
location = locations['location']

df = pd.DataFrame()
for loc in location:
    df_loc = []
    locationName = location[location.index(loc)]['locationName']
    df_loc = pd.Series(locationName).rename('locationName')
    for weather in loc['weatherElement']:
        if weather['description'] == '溫度':
            df_time = []
            df_val = []
            for time in pd.DataFrame.from_dict(weather)['time']:
                df_time.append(pd.DataFrame.from_dict(time, orient='index').T['dataTime'])
                df_val.append(pd.DataFrame.from_dict(time['elementValue'], orient='index').T['value'])
            df_time = pd.concat(df_time)
            df_val = pd.concat(df_val)
            df = pd.concat([df, pd.concat([df_loc, df_time, df_val], axis=1).reset_index(drop=True)]).reset_index(drop=True)
print(df)

    locationName                   dataTime value
0            鹽埕區  2019-06-28T12:00:00+08:00    33
1            鹽埕區  2019-06-28T15:00:00+08:00    33
2            鹽埕區  2019-06-28T18:00:00+08:00    32
3            鹽埕區  2019-06-28T21:00:00+08:00    30
4            鹽埕區  2019-06-29T00:00:00+08:00    30
5            鹽埕區  2019-06-29T03:00:00+08:00    29
6            鹽埕區  2019-06-29T06:00:00+08:00    28
7            鹽埕區  2019-06-29T09:00:00+08:00    31
8            鹽埕區  2019-06-29T12:00:00+08:00    32
9            鹽埕區  2019-06-29T15:00:00+08:00    32
10           鹽埕區  2019-06-29T18:00:00+08:00    31
11           鹽埕區  2019-06-29T21:00:00+08:00    30
12           鹽埕區  2019-06-30T00:00:00+08:00    29
13           鹽埕區  2019-06-30T03:00:00+08:00    28
14           鹽埕區  2019-06-30T06:00:00+08:00    28
15           鹽埕區  2019-06-30T09:00:00+08:00    31
16           鹽埕區  2019-06-30T12:00:00+08:00    32
17           鹽埕區  2019-06-30T15:00:00+08:00    32
18           鹽埕區  2019-06-30T18:00:00+08:00    31


In [22]:
df_loc_nrpt = df['locationName'].copy()
df_loc_nrpt.drop_duplicates(inplace = True)
df_loc_nrpt = df_loc_nrpt.reset_index(drop=True)
#print(df_loc_nrpt)

print(len(df_loc_nrpt))

38


In [23]:
# 2.請取出每一個地區所記錄的第一個時間點跟溫度 
df_loc_first = df.copy()
df_loc_first = df_loc_first.drop_duplicates(subset='locationName',keep='first').reset_index(drop=True)
print(df_loc_first)

   locationName                   dataTime value
0           鹽埕區  2019-06-28T12:00:00+08:00    33
1           鼓山區  2019-06-28T12:00:00+08:00    33
2           左營區  2019-06-28T12:00:00+08:00    33
3           楠梓區  2019-06-28T12:00:00+08:00    34
4           三民區  2019-06-28T12:00:00+08:00    32
5           新興區  2019-06-28T12:00:00+08:00    33
6           前金區  2019-06-28T12:00:00+08:00    33
7           苓雅區  2019-06-28T12:00:00+08:00    33
8           前鎮區  2019-06-28T12:00:00+08:00    33
9           旗津區  2019-06-28T12:00:00+08:00    32
10          小港區  2019-06-28T12:00:00+08:00    33
11          鳳山區  2019-06-28T12:00:00+08:00    34
12          林園區  2019-06-28T12:00:00+08:00    33
13          大寮區  2019-06-28T12:00:00+08:00    34
14          大樹區  2019-06-28T12:00:00+08:00    34
15          大社區  2019-06-28T12:00:00+08:00    34
16          仁武區  2019-06-28T12:00:00+08:00    33
17          鳥松區  2019-06-28T12:00:00+08:00    34
18          岡山區  2019-06-28T12:00:00+08:00    34
19          橋頭區  201

In [24]:
# 3. 請取出第一個地區所記錄的每一個時間點跟溫度
df_loc_1 = df.groupby(by='locationName', sort=False)
print(df_loc_1.get_group(df_loc_first['locationName'][0]))



   locationName                   dataTime value
0           鹽埕區  2019-06-28T12:00:00+08:00    33
1           鹽埕區  2019-06-28T15:00:00+08:00    33
2           鹽埕區  2019-06-28T18:00:00+08:00    32
3           鹽埕區  2019-06-28T21:00:00+08:00    30
4           鹽埕區  2019-06-29T00:00:00+08:00    30
5           鹽埕區  2019-06-29T03:00:00+08:00    29
6           鹽埕區  2019-06-29T06:00:00+08:00    28
7           鹽埕區  2019-06-29T09:00:00+08:00    31
8           鹽埕區  2019-06-29T12:00:00+08:00    32
9           鹽埕區  2019-06-29T15:00:00+08:00    32
10          鹽埕區  2019-06-29T18:00:00+08:00    31
11          鹽埕區  2019-06-29T21:00:00+08:00    30
12          鹽埕區  2019-06-30T00:00:00+08:00    29
13          鹽埕區  2019-06-30T03:00:00+08:00    28
14          鹽埕區  2019-06-30T06:00:00+08:00    28
15          鹽埕區  2019-06-30T09:00:00+08:00    31
16          鹽埕區  2019-06-30T12:00:00+08:00    32
17          鹽埕區  2019-06-30T15:00:00+08:00    32
18          鹽埕區  2019-06-30T18:00:00+08:00    31
19          鹽埕區  201